Polynomials are just another container.
bottom up ematching still seems ok
polynomials are ground from the term reqwriting perspective.
grobner guaranteed to terminate?
Also linear equalities

We can also do SMT guarded patterns.



linear inequalities?

Knuth bendix and grobner
https://leodemoura.github.io/files/synasc.pdf passmore de moura Superfluous S-polynomials in Strategy-Independent
Gr¨obner Bases
https://www.sciencedirect.com/science/article/pii/S0747717199902626  A Categorical Critical-pair Completion Algorithm
https://dl.acm.org/doi/pdf/10.1145/74540.74548  Knuth-Bendix procedure and Buchberger algorithm: a synthesis
https://link.springer.com/chapter/10.1007/978-3-0348-8800-4_9 Normalized Rewriting: A Unified View of Knuth-Bendix Completion and Gröbner Bases Computation
https://arxiv.org/abs/math/9812097  Applications of Rewriting Systems and Groebner Bases to Computing Kan Extensions and Identities Among Relations


https://jingnanshi.com/blog/groebner_basis.html
https://colab.research.google.com/github/bernalde/QuIP/blob/master/notebooks/Notebook%202%20-%20Groebner%20basis.ipynb hmm. quantum integer programming?
https://github.com/sumiya11/Groebner.jl?tab=readme-ov-file
msolve
https://www.philipzucker.com/computing-syzygy-modules-in-sympy/


Also polynomial factoring. Normalization of the poly could involve slapping the factors in the egraph.


In [1]:
import sympy as sp

In [ ]:
class UnionFind():
    def __init__(self):
        self.uf = []
    def makeset(self):
        uf = self.uf
        uf.append(len(uf))
        return len(uf) - 1
    def find(self,x):
        while self.uf[x] != x:
            x = self.uf[x]
        return x
    def union(self,x,y):
        x = self.find(x)
        y = self.find(y)
        self.uf[x] = y
        return y
    def __len__(self):
        return len(self.uf)
    def __repr__(self):
        return "UnionFind({})".format(self.uf)

EClass = int

from dataclasses import dataclass
from typing import Any, Dict

@dataclass(frozen=True)
class Lit():
    data: Any
    def rebuild(self):
        return self

@dataclass(frozen=True)
class ENode():
    head: Any
    args : tuple[EClass]
    def rebuild(self):
        return ENode(self.head, tuple(map(egraph.uf.find, self.args)))